In [186]:
import pandas as pd
import json

In [187]:
# abrindo o arquivo json do ecommerce
with open(r'ecommerce.json') as json_file:
    data = json.load(json_file)
data

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4675, 'relation': 'eq'},
  'max_score': 1,
  'hits': [{'_index': 'kibana_sample_data_ecommerce',
    '_id': 'Hn4v94YBwTR3CYJXEh--',
    '_score': 1,
    '_source': {'category': ["Men's Clothing"],
     'currency': 'EUR',
     'customer_first_name': 'Eddie',
     'customer_full_name': 'Eddie Underwood',
     'customer_gender': 'MALE',
     'customer_id': 38,
     'customer_last_name': 'Underwood',
     'customer_phone': '',
     'day_of_week': 'Monday',
     'day_of_week_i': 0,
     'email': 'eddie@underwood-family.zzz',
     'manufacturer': ['Elitelligence', 'Oceanavigations'],
     'order_date': '2023-04-03T09:28:48+00:00',
     'order_id': 584677,
     'products': [{'base_price': 11.99,
       'discount_percentage': 0,
       'quantity': 1,
       'manufacturer': 'Elitelligence',
       'tax_amount': 0,
       'product_id': 6283,
       'category': "Men'

In [251]:
# criando o dataframe com os dados desejados do ecommerce
hits = data['hits']['hits']
hits_list = []

for hit in hits:
    hit_data = hit['_source']
    hits_list.append(hit_data)
    
df_ecommerce = pd.DataFrame(hits_list)
df_ecommerce

,category,currency,customer_first_name,customer_full_name,customer_gender,customer_id,customer_last_name,customer_phone,day_of_week,day_of_week_i,...,products,sku,taxful_total_price,taxless_total_price,total_quantity,total_unique_products,type,user,geoip,event
0,[Men's Clothing],EUR,Eddie,Eddie Underwood,MALE,38,Underwood,,Monday,0,...,"[{'base_price': 11.99, 'discount_percentage': ...","[ZO0549605496, ZO0299602996]",36.98,36.98,2,2,order,eddie,"{'country_iso_code': 'EG', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
1,[Women's Clothing],EUR,Mary,Mary Bailey,FEMALE,20,Bailey,,Sunday,6,...,"[{'base_price': 24.99, 'discount_percentage': ...","[ZO0489604896, ZO0185501855]",53.98,53.98,2,2,order,mary,"{'country_iso_code': 'AE', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
2,"[Women's Shoes, Women's Clothing]",EUR,Gwen,Gwen Butler,FEMALE,26,Butler,,Sunday,6,...,"[{'base_price': 99.99, 'discount_percentage': ...","[ZO0374603746, ZO0272202722]",199.98,199.98,2,2,order,gwen,"{'country_iso_code': 'US', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
3,"[Women's Shoes, Women's Clothing]",EUR,Diane,Diane Chandler,FEMALE,22,Chandler,,Sunday,6,...,"[{'base_price': 74.99, 'discount_percentage': ...","[ZO0360303603, ZO0272002720]",174.98,174.98,2,2,order,diane,"{'country_iso_code': 'GB', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
4,"[Men's Clothing, Men's Accessories]",EUR,Eddie,Eddie Weber,MALE,38,Weber,,Monday,0,...,"[{'base_price': 59.99, 'discount_percentage': ...","[ZO0542505425, ZO0601306013]",80.98,80.98,2,2,order,eddie,"{'country_iso_code': 'EG', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[Women's Clothing, Women's Shoes]",EUR,Yasmine,Yasmine Daniels,FEMALE,43,Daniels,,Thursday,3,...,"[{'base_price': 7.99, 'discount_percentage': 0...","[ZO0646806468, ZO0238602386]",72.98,72.98,2,2,order,yasmine,"{'country_iso_code': 'SA', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
496,"[Men's Clothing, Men's Accessories]",EUR,Kamal,Kamal Dawson,MALE,39,Dawson,,Thursday,3,...,"[{'base_price': 13.99, 'discount_percentage': ...","[ZO0129101291, ZO0605606056]",38.98,38.98,2,2,order,kamal,"{'country_iso_code': 'TR', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
497,[Men's Clothing],EUR,Abd,Abd Potter,MALE,52,Potter,,Thursday,3,...,"[{'base_price': 6.99, 'discount_percentage': 0...","[ZO0551605516, ZO0457004570]",25.98,25.98,2,2,order,abd,"{'country_iso_code': 'EG', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}
498,"[Women's Clothing, Women's Shoes]",EUR,Gwen,Gwen Daniels,FEMALE,26,Daniels,,Thursday,3,...,"[{'base_price': 24.99, 'discount_percentage': ...","[ZO0067800678, ZO0030500305]",66.98,66.98,2,2,order,gwen,"{'country_iso_code': 'US', 'location': {'lon':...",{'dataset': 'sample_ecommerce'}


In [233]:
# criando o dataframe de regras baseado no arquivo rules.txt
columns = ['id_rule',
           'nm_rule',
           'ds_rule',
           'ds_rule_aux']

df_rules = pd.read_csv(r'rules.txt',
                       skiprows=1,
                       index_col=False,
                       names=columns)

# unindo os dados da coluna ds_rule que foram separados pela vírgula
df_rules['ds_rule'] = df_rules['ds_rule']+','+df_rules['ds_rule_aux']

# tratando as cláusulas do campo ds_rule para realização da query
df_rules['ds_rule'] = df_rules['ds_rule'].str.replace('in',' in ').str.replace('not',' not').str.replace('&',' and ')
df_rules['ds_rule'] = df_rules['ds_rule'].str.replace('`','').str.replace("'",'"')

# excluindo a coluna auxiliar usada na importação do dataframe
df_rules.drop('ds_rule_aux',
              axis=1,
              inplace=True)

df_rules

,id_rule,nm_rule,ds_rule
0,1,men_weekend,"day_of_week in [""Saturday"",""Sunday""] and custo..."
1,2,men_week,"day_of_week not in [""Saturday"",""Sunday""] and ..."
2,3,women_weekend,"day_of_week in [""Saturday"",""Sunday""] and custo..."
3,4,women_week,"day_of_week not in [""Saturday"",""Sunday""] and ..."


In [240]:
# adicionando as colunas do dataframe de regras ao dataframe de ecommerce
for _, rule in df_rules.iterrows():
    rule_condition = rule['ds_rule']
    rule_id = rule['id_rule']
    rule_name = rule['nm_rule']
    filtered_df_ecommerce = df_ecommerce.query(rule_condition)
    filtered_df_ecommerce['id_rule'] = rule_id
    filtered_df_ecommerce['nm_rule'] = rule_name
    if rule_id == 1:
        result_df = filtered_df_ecommerce
    else:
        result_df = pd.concat([result_df, filtered_df_ecommerce])

/var/folders/9p/4h56dlms78xd4zr6x0zdmm2c0000gn/T/ipykernel_70900/3286617238.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_ecommerce['id_rule'] = rule_id
/var/folders/9p/4h56dlms78xd4zr6x0zdmm2c0000gn/T/ipykernel_70900/3286617238.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_ecommerce['nm_rule'] = rule_name
/var/folders/9p/4h56dlms78xd4zr6x0zdmm2c0000gn/T/ipykernel_70900/3286617238.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [248]:
# criando o dataframe final apenas com as colunas desejadas
df = result_df[['order_date',
                'order_id',
                'id_rule',
                'nm_rule']].sort_index()
df

,order_date,order_id,id_rule,nm_rule
0,2023-04-03T09:28:48+00:00,584677,2,men_week
1,2023-04-02T21:59:02+00:00,584021,3,women_weekend
2,2023-04-02T22:32:10+00:00,584058,3,women_weekend
3,2023-04-02T22:58:05+00:00,584093,3,women_weekend
4,2023-03-27T03:48:58+00:00,574916,2,men_week
...,...,...,...,...
495,2023-03-30T17:12:29+00:00,579614,4,women_week
496,2023-03-09T17:03:50+00:00,551329,2,men_week
497,2023-04-06T08:28:19+00:00,588617,2,men_week
498,2023-04-06T06:36:00+00:00,588512,4,women_week


In [250]:
# exportando o dataframe final para .csv
df.to_csv('ecommerce_with_rules', 
          sep=',',
          index=False)